In [ ]:
import numpy as np
import os
import sys
import trimesh 
import pandas as pd
import plotly.graph_objects as go 
from plotly.subplots import make_subplots 

# To allow me to import the functiosn from other folders from the parent directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from File_Paths.file_paths import FEM_path, mesh_path 

# Import sample geometry
sample_geometry_mesh = trimesh.load(os.path.join(mesh_path, "411.obj"))
sample_geometry_FEM  = pd.read_csv(os.path.join(FEM_path, "411field.csv")).to_numpy()

# import data for the jet engine bracket test case
from Test_Cases.Jet_Engine_Bracket.JEB_Master_object import JEB_Master_object
from Functions.Point_Sampling.point_sampler import Point_Sampler  

JEB_data = JEB_Master_object(Normalize=False)

# import the domain data
domain_JEB = JEB_data.domain
domain_volume_JEB = JEB_data.domain_volume

# import centroids for interfaces
centroid_bolt1 = JEB_data.bolt1_centroid
centroid_bolt2 = JEB_data.bolt2_centroid
centroid_bolt3 = JEB_data.bolt3_centroid
centroid_bolt4 = JEB_data.bolt4_centroid
centroid_pinn1 = JEB_data.pinn1_centroid
centroid_pinn2 = JEB_data.pinn2_centroid

# import the geometric features
bolt_radius = JEB_data.bolt_interface_radius
bolt_depth = JEB_data.bolt_interface_depth
bolt_thickness = JEB_data.bolt_interface_thickness

pinn_radius = JEB_data.pinn_interface_radius
pinn_depth = JEB_data.pinn_interface_depth
pinn_thickness = JEB_data.pinn_interface_thickness
pinn_width = JEB_data.pinn_interface_width


counterbore_radius = JEB_data.counterbore_radius
counterbore_depth = JEB_data.counterbore_depth
counterbore_thickness = JEB_data.counterbore_interface_thickness

# import the force data
force_magnitude_vertical = JEB_data.force_magnitude_vertical  
force_direction_vertical = JEB_data.force_direction_vertical  
force_vector_vertical = JEB_data.force_vector_vertical       

force_magnitude_horizontal = JEB_data.force_magnitude_horizontal 
force_direction_horizontal = JEB_data.force_direction_horizontal 
force_vector_horizontal = JEB_data.force_vector_horizontal      

force_magnitude_diagonal = JEB_data.force_magnitude_diagonal 
force_direction_diagonal = JEB_data.force_direction_diagonal 
force_vector_diagonal = JEB_data.force_vector_diagonal       

**1. Scale the sample geometry using trimesh and use that as a reference for assesing the quality of my scaling functions:**

In [ ]:
#1. Zero-center the geometry 
bounds = sample_geometry_mesh.bounds
center = bounds.mean(axis=0)
sample_geometry_mesh.apply_translation(-center)

#2. Scale the geometry to fit in the unit cube
extents = bounds[1] - bounds[0]    
max_extent = np.max(extents)       
scale_factor = 2.0 / max_extent     
sample_geometry_mesh.apply_scale(scale_factor)  


In [ ]:
# Scale the geometry to fit within [-1,1] in all three axes  
domain_JEB_center = domain_JEB_center = np.array([
                                                    0.5*(domain_JEB[0] + domain_JEB[1]),
                                                    0.5*(domain_JEB[2] + domain_JEB[3]),
                                                    0.5*(domain_JEB[4] + domain_JEB[5]),
                                                ])

domain_JEB_extents = np.array([domain_JEB[1] - domain_JEB[0],
                               domain_JEB[3] - domain_JEB[2], 
                               domain_JEB[5] - domain_JEB[4]])

domain_JEB_max_extent = np.max(domain_JEB_extents)       
domain_JEB_scale_factor = 2.0 / domain_JEB_max_extent     

# shift the centroids of the interfaces to match the new domain 
centroid_bolt1_JEB = (centroid_bolt1 - domain_JEB_center) * domain_JEB_scale_factor
centroid_bolt2_JEB = (centroid_bolt2 - domain_JEB_center) * domain_JEB_scale_factor
centroid_bolt3_JEB = (centroid_bolt3 - domain_JEB_center) * domain_JEB_scale_factor
centroid_bolt4_JEB = (centroid_bolt4 - domain_JEB_center) * domain_JEB_scale_factor
centroid_pinn1_JEB = (centroid_pinn1 - domain_JEB_center) * domain_JEB_scale_factor
centroid_pinn2_JEB = (centroid_pinn2 - domain_JEB_center) * domain_JEB_scale_factor
centroids_JEB = np.array([centroid_bolt1_JEB, centroid_bolt2_JEB, centroid_bolt3_JEB, centroid_bolt4_JEB, centroid_pinn1_JEB, centroid_pinn2_JEB]) 

# Scale the geometric features to fit within the new domain 
bolt_radius = bolt_radius * domain_JEB_scale_factor
bolt_depth = bolt_depth * domain_JEB_scale_factor
bolt_thickness = bolt_thickness * domain_JEB_scale_factor

pinn_radius = pinn_radius * domain_JEB_scale_factor
pinn_depth = pinn_depth * domain_JEB_scale_factor
pinn_thickness = pinn_thickness * domain_JEB_scale_factor
pinn_width = pinn_width * domain_JEB_scale_factor


counterbore_radius = counterbore_radius * domain_JEB_scale_factor
counterbore_depth = counterbore_depth * domain_JEB_scale_factor
counterbore_thickness = counterbore_thickness * domain_JEB_scale_factor


In [ ]:

JEB_normalized = JEB_Master_object(Normalize = True)
JEB_normalized.create_interfaces()
point_sampler = Point_Sampler(JEB_normalized.domain, JEB_normalized.interfaces, num_points_domain = 1000000, num_points_interface= 20000)
points = next(point_sampler)


points_prohibited,_ = JEB_normalized.interfaces.is_inside_prohibited_region(points)
points_pinn_surface,_ = JEB_normalized.interfaces.is_on_pinn_interface(points) 
points_bolt_surface,_ = JEB_normalized.interfaces.is_on_bolt_interface(points)
points_thickness,_ = JEB_normalized.interfaces.is_inside_interface_thickness(points) 

In [ ]:
print("bolt depth", JEB_normalized.bolt_interface_depth)
print("pinn depth", JEB_normalized.pinn_interface_depth)
print("counterbore_depth", JEB_normalized.counterbore_depth) 

In [ ]:
# Plot to confirm that the mesh was centered correctly
fig = go.Figure()

inside_pts         = points_prohibited
surface_pts        = np.concatenate((points_bolt_surface, points_pinn_surface), axis=0)
thickness_pts      = points_bolt_surface 


fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=inside_pts[:,0], y=inside_pts[:,1], z=inside_pts[:,2],
    mode='markers',
    marker=dict(size=2, color='red', opacity=0),
    name='Prohibited core'
))

fig.add_trace(go.Scatter3d(
    x=thickness_pts[:,0], y=thickness_pts[:,1], z=thickness_pts[:,2],
    mode='markers',
    marker=dict(size=2, color='purple', opacity=0),
    name='Bolt-thickness shell'
))

fig.add_trace(go.Mesh3d(
    x=sample_geometry_mesh.vertices[:,0], y=sample_geometry_mesh.vertices[:,1], z=sample_geometry_mesh.vertices[:,2],
    i=sample_geometry_mesh.faces[:,0], j=sample_geometry_mesh.faces[:,1], k=sample_geometry_mesh.faces[:,2],
    opacity=1,
    name='Mesh',
    color='blue'
))


fig.update_layout(
    scene=dict(
        xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
        aspectmode='data'
    ),
    title="" \
    "Scaled and centered mesh of part 411"
)
fig.show()

In [ ]:

mesh_unscaled = trimesh.load(os.path.join(mesh_path, "411.obj"))

verts_u = mesh_unscaled.vertices
faces_u = mesh_unscaled.faces

fig_unscaled = go.Figure()
fig_unscaled.add_trace(go.Mesh3d(
    x=verts_u[:, 0], y=verts_u[:, 1], z=verts_u[:, 2],
    i=faces_u[:, 0], j=faces_u[:, 1], k=faces_u[:, 2],
    opacity=1.0,
    name='Mesh (unscaled)',
    color='blue'
))

fig_unscaled.update_layout(
    scene=dict(
        xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
        aspectmode='data'
    ),
    title='Original (unscaled) mesh of part 411'
)

fig_unscaled.show()